In [21]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [22]:
file_path = "./jsonl/batch_qKddqZA9J4N7m75McMwbjjrE_output.jsonl"
gt_data = pd.read_json(f"./images/configurations.json")

In [23]:
# Dictionary to map custom_id to message content
custom_id_to_content = {}

# Open the file and read line by line
with open(file_path, "r") as file:
    for line in file:
        json_obj = json.loads(line)
        # Extract custom_id and message content
        custom_id = json_obj.get("custom_id")
        message_content = json_obj["response"]["body"]["choices"][0]["message"][
            "content"
        ]
        # Map custom_id to message content
        if custom_id:
            custom_id_to_content[custom_id.replace("uid__", "")] = message_content

# Print the dictionary to verify
print(len(custom_id_to_content))

148


In [24]:
gt_data = (
    gt_data.T
)  # use indexx as a new column in the dataframe and called it image_path
gt_data = gt_data.reset_index()
gt_data = gt_data.rename(columns={"index": "image_path"})

# ad a new column to gt_data with content of  custom_id_to_content (match key with image_path)
gt_data["message"] = gt_data["image_path"].map(custom_id_to_content)

In [25]:
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness,message
0,image_0_thickness_2.png,3,1,"[red, blue]",2,There is one intersection point where the red ...
1,image_0_thickness_3.png,3,1,"[red, blue]",3,There are 2 line intersections in the image.
2,image_0_thickness_4.png,3,1,"[red, blue]",4,There are two line intersections in the image.
3,image_1_thickness_2.png,3,2,"[red, blue]",2,There are 4 line intersections in the image.
4,image_1_thickness_3.png,3,2,"[red, blue]",3,There are two line intersections in the image.
...,...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3,The image shows two lines intersecting. There ...
146,image_48_thickness_4.png,3,2,"[red, blue]",4,There are three line intersections in the imag...
147,image_49_thickness_2.png,3,0,"[red, blue]",2,There are two intersections among the lines in...
148,image_49_thickness_3.png,3,0,"[red, blue]",3,There is one line intersection in the image wh...


In [26]:
# remove nan
gt_data = gt_data.dropna()

In [27]:
import re


# Function to extract numbers from text, including zero represented as text
def extract_number(text):
    # Check for phrases that imply zero intersections
    print(text)
    if (
        "no intersections" in text
        or "zero line intersections" in text
        or "no line" in text
    ):
        return 0
    # First try to find digits
    match = re.search(r"\b\d+\b", text)
    if match:
        return int(match.group())
    # If no digits, look for text numbers (one, two, three, etc.)
    text_to_num = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5}
    for word, num in text_to_num.items():
        if word in text:
            return num
    return None


# Apply the function to the 'message' column
gt_data["extracted_number"] = gt_data["message"].apply(extract_number)
gt_data[["image_path", "message", "extracted_number"]]

There is one intersection point where the red and blue lines cross each other in the image.
There are 2 line intersections in the image.
There are two line intersections in the image.
There are 4 line intersections in the image.
There are two line intersections in the image.
There are three line intersections in the image.
There are two intersections in the image. The blue line intersects each of the red lines once.
There are two intersections in the image. The two red lines intersect with the blue line at two different points.
There is one line intersection in the image.
There are two line intersections in the image.
There are two line intersections in the image.
There are three line intersections in the image. The red line intersects each of the two blue lines once, and the two blue lines intersect each other once.
There are three line intersections in the image.
There are two intersection points in the image where the red lines intersect the blue line.
There are two line intersectio

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_19387/469737310.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gt_data["extracted_number"] = gt_data["message"].apply(extract_number)


,image_path,message,extracted_number
0,image_0_thickness_2.png,There is one intersection point where the red ...,1
1,image_0_thickness_3.png,There are 2 line intersections in the image.,2
2,image_0_thickness_4.png,There are two line intersections in the image.,2
3,image_1_thickness_2.png,There are 4 line intersections in the image.,4
4,image_1_thickness_3.png,There are two line intersections in the image.,2
...,...,...,...
145,image_48_thickness_3.png,The image shows two lines intersecting. There ...,1
146,image_48_thickness_4.png,There are three line intersections in the imag...,3
147,image_49_thickness_2.png,There are two intersections among the lines in...,2
148,image_49_thickness_3.png,There is one line intersection in the image wh...,1


In [28]:
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness,message,extracted_number
0,image_0_thickness_2.png,3,1,"[red, blue]",2,There is one intersection point where the red ...,1
1,image_0_thickness_3.png,3,1,"[red, blue]",3,There are 2 line intersections in the image.,2
2,image_0_thickness_4.png,3,1,"[red, blue]",4,There are two line intersections in the image.,2
3,image_1_thickness_2.png,3,2,"[red, blue]",2,There are 4 line intersections in the image.,4
4,image_1_thickness_3.png,3,2,"[red, blue]",3,There are two line intersections in the image.,2
...,...,...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3,The image shows two lines intersecting. There ...,1
146,image_48_thickness_4.png,3,2,"[red, blue]",4,There are three line intersections in the imag...,3
147,image_49_thickness_2.png,3,0,"[red, blue]",2,There are two intersections among the lines in...,2
148,image_49_thickness_3.png,3,0,"[red, blue]",3,There is one line intersection in the image wh...,1


In [29]:
# count none or na in gt_data extracted_number
gt_data["extracted_number"].isna().sum()
# show their value in the message (same row)
gt_data[gt_data["extracted_number"].isna()]["message"]

Series([], Name: message, dtype: object)

In [30]:
# Cast num_intersections to integer
gt_data["num_intersections"] = gt_data["num_intersections"].astype(int)

# Cast extracted_number to integer (it should already be integer from the extraction function, but just to ensure consistency)
gt_data["extracted_number"] = gt_data["extracted_number"].astype(int)

# Compare num_intersections and extracted_number to check if they are the same
gt_data["is_correct"] = gt_data["num_intersections"] == gt_data["extracted_number"]

# Display the DataFrame with the new column
gt_data[["image_path", "num_intersections", "extracted_number", "is_correct"]]

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_19387/31822554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gt_data["num_intersections"] = gt_data["num_intersections"].astype(int)
/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_19387/31822554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gt_data["extracted_number"] = gt_data["extracted_number"].astype(int)
/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_19387/31822554.py:8: SettingWithCopyWarning: 
A value is tr

,image_path,num_intersections,extracted_number,is_correct
0,image_0_thickness_2.png,1,1,True
1,image_0_thickness_3.png,1,2,False
2,image_0_thickness_4.png,1,2,False
3,image_1_thickness_2.png,2,4,False
4,image_1_thickness_3.png,2,2,True
...,...,...,...,...
145,image_48_thickness_3.png,2,1,False
146,image_48_thickness_4.png,2,3,False
147,image_49_thickness_2.png,0,2,False
148,image_49_thickness_3.png,0,1,False


In [31]:
gt_data["is_correct"].value_counts()

is_correct
False    105
True      43
Name: count, dtype: int64

In [32]:
100 * np.mean(gt_data["is_correct"])

29.054054054054053

In [33]:
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness,message,extracted_number,is_correct
0,image_0_thickness_2.png,3,1,"[red, blue]",2,There is one intersection point where the red ...,1,True
1,image_0_thickness_3.png,3,1,"[red, blue]",3,There are 2 line intersections in the image.,2,False
2,image_0_thickness_4.png,3,1,"[red, blue]",4,There are two line intersections in the image.,2,False
3,image_1_thickness_2.png,3,2,"[red, blue]",2,There are 4 line intersections in the image.,4,False
4,image_1_thickness_3.png,3,2,"[red, blue]",3,There are two line intersections in the image.,2,True
...,...,...,...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3,The image shows two lines intersecting. There ...,1,False
146,image_48_thickness_4.png,3,2,"[red, blue]",4,There are three line intersections in the imag...,3,False
147,image_49_thickness_2.png,3,0,"[red, blue]",2,There are two intersections among the lines in...,2,False
148,image_49_thickness_3.png,3,0,"[red, blue]",3,There is one line intersection in the image wh...,1,False


In [34]:
# Assuming 'gt_data' DataFrame has columns 'line_thickness', 'num_intersections', and 'extracted_number'

# Convert line_thickness to an appropriate numeric type if necessary
gt_data["line_thickness"] = gt_data["line_thickness"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = gt_data.groupby("line_thickness").apply(
    lambda df: (df["num_intersections"] == df["extracted_number"]).mean()
)

# Print the accuracy by line thickness
# print(accuracy_by_thickness
#
# #)
# multiply by 100 and round 2 decimal
accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_19387/1027132002.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gt_data["line_thickness"] = gt_data["line_thickness"].astype(int)
/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_19387/1027132002.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = gt_data.groupby("line_thickness").apply(


In [35]:
print(accuracy_by_thickness.to_latex())

\begin{tabular}{lr}
\toprule
 & 0 \\
line_thickness &  \\
\midrule
2 & 33.000000 \\
3 & 24.000000 \\
4 & 31.000000 \\
\bottomrule
\end{tabular}



# Export

In [37]:
gt_data["Model"] = ["gpt-4o"] * len(gt_data)
# rename message to model_output_raw
gt_data.rename(columns={"message": "model_output_raw"}, inplace=True)
gt_data.to_pickle("./data/gpt-4o.pkl")

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_19387/3718921116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gt_data["Model"] = ["gpt-4o"] * len(gt_data)
/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_19387/3718921116.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gt_data.rename(columns={"message": "model_output_raw"}, inplace=True)
